In [ ]:
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

In [ ]:
data = 'data/agri/Wisconsin_1960+2009_Annual+Crop+Data.csv'
wiag = pd.read_csv(data)
wiag['DataItem'] = wiag['Data Item']
wiag['year'] = wiag['Year']
wiag = wiag[~wiag.Value.str.contains('D')]
wiag = wiag[~wiag.Value.str.contains('X')]
wiag.Value = wiag.Value.str.strip()
wiag.Value = wiag.Value.str.replace(r'\(Z\)', '0')
wiag.Value = wiag.Value.str.replace(',', '')
wiag.Value = pd.to_numeric(wiag.Value)
peracre = wiag[wiag.DataItem.str.contains("/ ACRE")]
comm = list(set(peracre.Commodity))
crops = {}
for c in comm:
    rows = peracre[peracre.Commodity == c]
    rows = rows[rows.DataItem.str.contains(c + ' - YIELD')]
    crops[c] = rows
comm

In [ ]:
data = 'data/wx/wxsummary.csv'
wx = pd.read_csv(data)
wiwx = wx[wx.state == 'WI']
modata = wiwx.loc[:, 'may':'sep'].mean(axis=1)
wiwx = wiwx.copy()
wiwx['mean'] = modata
prcp = wiwx[wiwx['type'] == 'prcp']
prcp = prcp[['year', 'mean']]
tavg = wiwx[wiwx['type'] == 'tavg']
tavg = tavg[['year', 'mean']]
tavg['mean'].plot()

In [ ]:
rows = []
for key in comm:
    merged = pd.merge(crops[key], prcp, on='year')
    if len(merged) == 0:
        continue
    row = [key, len(merged), merged.Value.corr(merged['mean'])]
    row.append(abs(row[2]))
    rows.append(row)
    
corrs = pd.DataFrame(rows, columns=['crop', 'years', 'corr', 'abscorr'])
corrs.sort_values('abscorr')


In [ ]:
merged = pd.merge(crops['HAYLAGE'], prcp, on='year')
model = linear_model.LinearRegression()

meanprcp = [[x] for x in merged['mean']]
model.fit(meanprcp, merged.Value)

m = model.coef_[0]
b = model.intercept_
plt.scatter(merged['mean'], merged.Value)
plt.plot(merged['mean'], m * merged['mean'] + b, color='red')
plt.show()


In [ ]:
rows = []
for key in comm:
    merged = pd.merge(crops[key], tavg, on='year')
    if len(merged) == 0:
        continue
    row = [key, len(merged), merged.Value.corr(merged['mean'])]
    row.append(abs(row[2]))
    rows.append(row)
    
corrs = pd.DataFrame(rows, columns=['crop', 'years', 'corr', 'abscorr'])
corrs.sort_values('abscorr')


In [ ]:
merged = pd.merge(crops['SOYBEANS'], prcp, on='year')
model = linear_model.LinearRegression()

meanprcp = [[x] for x in merged['mean']]
model.fit(meanprcp, merged.Value)

m = model.coef_[0]
b = model.intercept_
plt.scatter(merged['mean'], merged.Value)
plt.plot(merged['mean'], m * merged['mean'] + b, color='red')
plt.show()